In [ ]:
!pip install finesse
import numpy as np
import matplotlib.pyplot as plt
import finesse
#from finesse.analysis.actions import RunLocks
finesse.configure(plotting=True)

In [ ]:
kat = finesse.Model()
kat.parse(
    """
    ###########################################################################
    ###   Variables
    ###########################################################################
    var Larm 2999.8
    var Lprc 12
    var Lsrc 12
    var Mtm  40
    var prmT 0.04835
    var itmT 0.01377
    var srmT 0.325
    var lmichx 5.3662
    var lmichy 5.244
    var f1 6270777
    var f2 9*f1
    var f3 4/3*f1

    ###########################################################################
    ###   Input optics
    ###########################################################################
    l L0 125
    s l_mod1 L0.p1 eo1.p1
    mod eo1 f1 0.1 order=3
    s l_mod2 eo1.p2 eo2.p1
    mod eo2 f2 0.1 order=3
    s l_mod3 eo2.p2 eo3.p1
    mod eo3 f3 0.1 order=3
    s l_in eo3.p2 prm.p1 L=53
    #Power recycling mirror
    m prm T=prmT L=30u Rc=-1430
    s l_prc prm.p2 bs.p1 L=Lprc
    # Central beamsplitter
    bs bs T=0.5 L=0 alpha=45

    ###########################################################################
    ###   X arm
    ###########################################################################
    s lx bs.p3 itmx.p1 L=lmichx
    m itmx T=itmT L=37.5u phi=90 Rc=-1424.6
    s LX itmx.p2 etmx.p1 L=Larm
    m etmx T=5u L=37.5u phi=90 Rc=1695.0

    ###########################################################################
    ###   Y arm
    ###########################################################################
    s ly bs.p2 itmy.p1 L=lmichy
    m itmy T=itmT L=37.5u phi=0 Rc=-1424.5
    s LY itmy.p2 etmy.p1 L=Larm
    m etmy T=5u L=37.5u phi=0.0 Rc=1696.0

    ###########################################################################
    ###   Signal Recyling Cavity
    ###########################################################################
    s l_src bs.p4 srm.p1 L=Lsrc
    m srm T=srmT L=30u Rc=1430.0

    ###########################################################################
    ###   Cavities
    ###########################################################################
    # Arms
    cav cavX itmx.p2.o priority=3
    cav cavY itmy.p2.o priority=3
    # PRC
    cav cavPRW prm.p2.o via=itmx.p1.i priority=2
    cav cavPRN prm.p2.o via=itmy.p1.i priority=2
    # SRC
    cav cavSRW srm.p1.o via=itmx.p1.i priority=1
    cav cavSRN srm.p1.o via=itmy.p1.i priority=1

    ###########################################################################
    ###   Degrees of Freedom, Readouts, and Locks
    ###########################################################################
    dof DARM etmx.dofs.z -1 etmy.dofs.z +1
    dof CARM etmx.dofs.z +1 etmy.dofs.z +1
    dof MICH itmx.dofs.z -1 etmx.dofs.z -1 itmy.dofs.z +1 etmy.dofs.z +1
    dof PRCL prm.dofs.z +1
    dof SRCL srm.dofs.z -1

    readout_rf rd_sr_f1 srm.p2.o f=eo1.f phase=0 output_detectors=True
    readout_rf rd_pr_f1 prm.p1.o f=eo1.f phase=0
    readout_rf rd_pr_f2 prm.p1.o f=eo2.f phase=0
    readout_rf rd_pr_f3 prm.p1.o f=eo3.f phase=0

    """
)